In [ ]:
import re
import os
import json
import datetime as dt
import pandas as pd

In [ ]:
def process_date(start_date, end_date):
  """Create helper function to use for starting, ending date of group buy
      Case1: has month day, year in the string
      Case2: Has only month and day -> has to infer from the other dates
      Case3: Sold out      
  """
  
  ymd_pattern = '(\D*)\s*(\d{1,2})\s*(?:rd|th|st|nd)*\,*\s*(\d{4})*'
  
  start_tup = re.search(ymd_pattern, start_date)
  start_m, start_d, start_y = start_tup.group(1), start_tup.group(2), start_tup.group(3)
  
  end_y = None
  end_d = None
  end_m = None
  
  if end_date != 'sold out': 
    end_tup = re.search(ymd_pattern, end_date)
    
    if end_tup is not None: 
      end_m, end_d, end_y = end_tup.group(1), end_tup.group(2), end_tup.group(3)    
    if end_y is None and start_y is not None: end_y = start_y
  
  if start_y is None and end_y is not None: start_y = end_y  
  out_start_date = dt.datetime.strptime(f"{start_d.strip()}-{start_m.strip()}-{start_y.strip()}", '%d-%B-%Y')
  
  #print(end_d, end_m, end_y)
  
  if end_date == 'sold out': 
    out_end_date = end_date
  elif end_d is None or end_d == '' or end_m is None or end_m == '':
    out_end_date = 'sold out'
  else:
    out_end_date = dt.datetime.strptime(f"{end_d.strip()}-{end_m.strip()}-{end_y.strip()}", '%d-%B-%Y')
  
  return out_start_date, out_end_date

In [ ]:
def process_price(price, mode='mid'):
  """Helper Function used to process price column in case of varying price, e.g.: 30-55
      mode = 'mid': calculate the mid price
      mode = 'min': get the lower bound price
      mode = 'max': get the upper bound price
  """
  
  print(price)
  try:
    price = price.replace('---', '') # Hack for some case
    output_price = float(price)    
  except:
    low_price = float(price[:price.find('-')])
    high_price = float(price[price.find('-'):])
    if mode =='mid': 
      output_price = (low_price + high_price) * 0.5
    elif mode == 'min':
      output_price = low_price
    elif mode == 'max':
      output_price = high_price
      
  return output_price

In [ ]:
scraped_data = []
title_screen = []
for file in os.listdir('data'):
  #scraped_data = json.load(open("scrapped_data.json", "r"))
  if file.endswith('.json'):
    scrp = json.load(open(os.path.join('data', file), 'r'))
    scraped_data.append(scrp)
    title_screen.append(scrp['title'])
len(set(title_screen))

### Contruct the Dataframe

In [ ]:
# Have a separate cell for the shipping date

#ship_date_pattern = '(?:est\.\s*shipping\s*date|est.\s*fulfillment\s*date):\s*\*\*(.*?)\*\*---'
#ship_date_pattern = '(?:.*?):\s*\*\*(.*?)\*\*---'
# ship_date_pattern = \
# """(?:est\.?\s*shipping\s*date|est.\s*fulfillment\s*date|est\.?\s*delivery\s*date|est\.?\s*delivering\s*date|estimated\s*\ship)
# (?:from\s*\w*\s*)?:\s*\*\*(.*?)\*\*---"""


# There's probably better method for this, like using word stemming
ship_date_pattern = re.compile((
  '(?:'
    '(?:'
      '(?:est|estimated|est.|estimate)\s*'
      '|'
      '(?:eta)\s*'
    ')'
    '(?:delivery|delivering|ship|shiping|shipping|fulfilment|fulfillment)?\s*'
    '(?:date)?\s*'
    '(?:from\s*\w*\s*)?'
  '):\s*\*?\*?(.*?)\*?\*?---'
))
  

In [ ]:
#price_pattern = '(?:(price|prices):)(\w*):\s*\*\*\s*\$?(\d*\-*\d*)\*\*'
#price_pattern = '(?:\*\*(?:price:|prices:)?\*\*)?\s*((\w*\s*):\s*\*\*\s*\$?(\d*\-*\d*)\*\*)'
price_pattern = '(?:\*\*(?:price:|prices:)?\*\*)?\s*(([^:]*):\s*\*\*\s*\$?(\d*\-*\d*)\*\*)'
title_pattern = '\[(?:GB|Pre-order)\]\s*(.*)\s*//' #Trying not to lower case to preserve some kb title that has uppercase
date_pattern = '//(.*)-(.*)'
vendor_pattern = '(\w*:\s*)\[(\w*)\]'

type_list = []
price_list = []
title_list = []
start_list = []
end_list = []
country_list = []
vendor_list = []
ship_date_list = []
sticky_comment_list = [] #Does not really need if the code is super stable

for scrp in scraped_data:
  price_tup = re.findall(price_pattern, scrp['sticky_comment'].replace('\n', '').lower())
  try:
    title = re.findall(title_pattern, scrp['title'].replace('\n', ''))[0] #Title should match only 1 string  
  except:
    title = scrp['title'].replace('[GB]', '').strip()
  try:
    ship_date = re.findall(ship_date_pattern, scrp['sticky_comment'].replace('\n', '').lower())[0]
  except:
    ship_date = None
      
  # first index is for tuple, second index is for the starting/ending date
  try:
    start_date = re.findall(date_pattern, scrp['title'].replace('\n', ''))[0][0].strip().lower()
    end_date = re.findall(date_pattern, scrp['title'].replace('\n', ''))[0][1].strip().lower()
    start_date, end_date = process_date(start_date, end_date)
  except:
    start_date = None
    end_date = None
  
  vendor_tup = re.findall(vendor_pattern, scrp['sticky_comment'].replace('\n', ''))
  
  #print(len(price_tup), len(vendor_tup))
  if len(price_tup) == 0: print(title)
  for p_tup in price_tup: 
    for v_tup in vendor_tup:      
      
      type_list.append(p_tup[1]) #group 1 and group 2 of the match groups
      price_list.append(p_tup[2])
      
      country_list.append(v_tup[0].replace(':', ''))
      vendor_list.append(v_tup[1])

      # These 4, 5 need to be duplicated across price/country
      title_list.append(title)
      start_list.append(start_date)
      end_list.append(end_date)
      ship_date_list.append(ship_date)
      sticky_comment_list.append(scrp['sticky_comment'].replace('\n', ''))

df = pd.DataFrame({'title': title_list, 'type': type_list, 'price': price_list, 'start_date': start_list, 'end_date':end_list,
                   'country': country_list, 'vendor': vendor_list, 'ship_date': ship_date_list, 'sticky_comment': sticky_comment_list
                  })
#df['price'] = df['price'].apply(lambda x: process_price(x, mode='mid'))
#df
df[df['price']=='']
df['title'].value_counts()
#print(df[df['ship_date'].isna()]['sticky_comment'].count())
#df[df['ship_date'].isna()]['sticky_comment'].sample(50)

In [ ]:
df.shape, df.drop_duplicates().shape

### Try to Extract the type (Keyboard vs Keycap)

In [ ]:
def sep_mech_product(row):
  """Drop duplicates by title and then pass in row of the dataframe for the separation"""
  # Need heuristic to separate (unless use some model to separate pictures)    

  # Let the rest falls down to keybaord, the hard part would be to differentiate keybard and switch
  # Switch may be look from linear/tactile in the description or something, as well as look at the title
  
  # keycap title may contains GMK, ePBT, MW, Domikey, DCS, SA
  for keycap_brand in ['gmk', 'epbt', 'mw', 'domikey', 'dcs', 'sa', 'osume']:
    if keycap_brand in row['title'].lower():      
      return 'keycap'
  
  # keycap type may include Alphas, Numpad, Spacebars, keycap
  for type_kb in row['agg_type']: 
    if 'alphas' in type_kb.lower() or 'numpad' in type_kb.lower() or 'spacebar' in type_kb.lower() or 'artisan' in type_kb.lower() \
    or 'num' in type_kb.lower():
      return 'keycap'
  
  # If is keyboard should have at least pcb or plate option
  for type_kb in row['agg_type']: 
    if 'pcb' in type_kb.lower() or 'plate' in type_kb.lower() or 'fr4' in type_kb.lower():
      return 'keyboard'
    
  # Check for 6x, 7x, 8x, tkl, 9x, 4x in name to flag for the keyboard
  kb_sizes = [str(i) for i in range(60, 90)] + [str(i) for i in range(40, 50)] + ['tkl']
  for kb_size in kb_sizes:
    if kb_size in row['title'].lower():      
      return 'keyboard'
    
  # Check for keyboard in title
  if 'keyboard' in row['title'].lower():
    return 'keyboard'
  
  # Check for deskmat in title
  if 'deskmat' in row['title'].lower() or 'desk mat' in row['title'].lower():    
    return 'deskmat'
  
  # Check for keycap in title
  if 'keycap' in row['title'].lower() or 'key cap' in row['title'].lower():    
    return 'deskmat'
  
  # If has switch in the type list should be switch
  for type_kb in row['agg_type']:
    if 'switch' in type_kb.lower():      
      return 'switch'
  
  # If pass everything, flag as keyboard

In [ ]:
# Need to combine the type for the
df['agg_type'] = df.groupby('title')['type'].transform(lambda x: [x.tolist()]*len(x))
# type count will be useful to see if there is only one mode of buying
df['agg_type_count'] = df.groupby('title')['type'].transform(lambda x: [set(x.tolist())]*len(x))
df_prod = df.drop_duplicates(['title']).copy()

# Get the product type and join back
df_prod['product'] = df_prod.apply(lambda x: sep_mech_product(x), axis=1)
df_prod = df_prod[['title', 'product']].copy()
df_prod['title'] = df_prod['title'].astype(str)
df['title'] = df['title'].astype(str)
df = df.merge(df_prod, on=['title'], how='left')

### Get Sum of All Price (per title per distributor), 
also get base minimal price to buy (check for 'base' or 'it' keyword or so)

In [ ]:
def check_base_price(type_kb):
  for base_type in ['base', 'kit', 'alpha', 'alphas']:
    if type_kb.lower() == base_type: return True
  return False

In [ ]:
# Get Sum of the price per title per studio per country
df['sum_all_price'] = df.groupby(['title', 'vendor', 'country'])['price'].transform('sum')

# get base/kit price
df_base = df.loc[df['type'].apply(lambda x: check_base_price(x)), :].copy()
df_base = df_base[['title', 'price']].copy().rename(columns={'price': 'base_price'})
df = df.merge(df_base, on=['title'], how='left')

# In case there is only one mode of buying, count that as base price
df.loc[df['agg_type_count']==1, 'base_price'] = df.loc[df['agg_type_count']==1, 'price']

In [ ]:
df[df['base_price'].isna()]
df[df['title'].str.contains('GMK Olivia')]

### Distribution of price

### Distribution of Keyboard Type 

### Distribution of the Studio 

### Visualization of number of active group buy in a Year (maybe a stack chart of avaialble group buys/ price need to buy everything, etc.)

### Time from Group Buy to Release (as per the initial announcement data)